In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import lightning as L
from torch.utils.data import DataLoader, TensorDataset


In [45]:
class LSTMTrain(L.LightningModule):
    def __init__(self):
        super().__init__()

        self.lstm = nn.LSTM(input_size=1, hidden_size=1)
    
    def forward(self, input):
        input_trans = input.transpose(0,1)
        input_trans = input_trans.unsqueeze(-1)

        lstm_out, temp = self.lstm(input_trans)
        prediction = lstm_out[-1]
        return prediction.squeeze()
    
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.1)
    
    def training_step(self, batch, batch_idx):
        input_i, label_i = batch
        output_i = self.forward(input_i)
        label_i = label_i.squeeze()   
        loss = F.mse_loss(output_i, label_i)
        self.log('train_loss', loss)

        if label_i.item() == 0:
            self.log('out_0', output_i)
        else:
            self.log('out_1', output_i)
            
        return loss

In [46]:
model = LSTMTrain()
print("Let's compare the observed and predicted values: ")
print("Company A: Observed = 0, Predicted =", model(torch.tensor([[0., 0.5, 0.25, 1.]])).detach().item())
print("Company B: Observed = 1, Predicted =", model(torch.tensor([[1., 0.5, 0.25, 1.]])).detach().item())

Let's compare the observed and predicted values: 
Company A: Observed = 0, Predicted = -0.44594836235046387
Company B: Observed = 1, Predicted = -0.4488987326622009


In [49]:
model = LSTMTrain()
inputs = torch.tensor([[0., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1]])
labels = torch.tensor([0.0, 1.0])

dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

trainer = L.Trainer(max_epochs=250, log_every_n_steps=2)
trainer.fit(model, train_dataloaders=dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name | Type | Params | Mode 
--------------------------------------
0 | lstm | LSTM | 16     | train
--------------------------------------
16        Trainable params
0         Non-trainable params
16        Total params
0.000     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode


Epoch 249: 100%|██████████| 2/2 [00:00<00:00, 97.13it/s, v_num=1] 

`Trainer.fit` stopped: `max_epochs=250` reached.


Epoch 249: 100%|██████████| 2/2 [00:00<00:00, 81.29it/s, v_num=1]


In [54]:
print("Let's compare the observed and predicted values: ")
print("Company A: Observed = 0, Predicted =", round(model(torch.tensor([[0., 0.5, 0.25, 1.]])).detach().item(), 4))
print("Company B: Observed = 1, Predicted =", round(model(torch.tensor([[1., 0.5, 0.25, 1.]])).detach().item(), 4))

Let's compare the observed and predicted values: 
Company A: Observed = 0, Predicted = 0.0
Company B: Observed = 1, Predicted = 0.9836
